<a href="https://colab.research.google.com/github/Yosoy1408/skills-introduction-to-github/blob/main/CasasParaTodosBot_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
# Instalación de todas las dependencias necesarias
!pip install python-telegram-bot==13.7
!pip install pandas numpy matplotlib
!pip install Pillow
!pip install requests
!pip install firebase-admin
!pip install python-dotenv
!pip install schedule
!pip install emoji
!pip install plotly
!pip install seaborn
!pip install opencv-python-headless
!pip install python-dateutil
!pip install pytz

In [45]:
import os
import json
import sqlite3
import logging
import random
import time
import schedule
import emoji
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import pytz
from PIL import Image, ImageDraw, ImageFont
from io import BytesIO
from telegram import Update, InlineKeyboardButton, InlineKeyboardMarkup, ParseMode
from telegram.ext import (
    Updater,
    CommandHandler,
    CallbackQueryHandler,
    CallbackContext,
    MessageHandler,
    Filters,
    ConversationHandler
)
from google.colab import drive
import IPython
from threading import Thread

In [47]:
# Token del bot
TOKEN = "7643294498:AAEJM1ftebsJmN1Jt2-lXkKA3l7bMcd7n0Q"

# Información del creador
CREATOR_INFO = {
    "name": "Sebastián Sánchez Navarro",
    "email": "inverydesar@gmail.com",
    "project": "Casas para Todos"
}

# Configuración de logging
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    level=logging.INFO
)
logger = logging.getLogger(__name__)

In [49]:
def init_database():
    conn = sqlite3.connect('dreambuilder.db')
    c = conn.cursor()

    # Tabla de usuarios
    c.execute('''CREATE TABLE IF NOT EXISTS users
                 (user_id INTEGER PRIMARY KEY,
                  username TEXT,
                  first_name TEXT,
                  last_name TEXT,
                  level INTEGER DEFAULT 1,
                  coins INTEGER DEFAULT 0,
                  experience INTEGER DEFAULT 0,
                  last_daily TIMESTAMP,
                  houses_built INTEGER DEFAULT 0,
                  total_donations REAL DEFAULT 0,
                  referral_code TEXT UNIQUE,
                  referred_by INTEGER,
                  join_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP)''')

    # Tabla de donaciones
    c.execute('''CREATE TABLE IF NOT EXISTS donations
                 (donation_id INTEGER PRIMARY KEY AUTOINCREMENT,
                  user_id INTEGER,
                  amount REAL,
                  timestamp TIMESTAMP,
                  payment_method TEXT,
                  status TEXT,
                  transaction_id TEXT UNIQUE,
                  FOREIGN KEY (user_id) REFERENCES users(user_id))''')

    # Tabla de logros
    c.execute('''CREATE TABLE IF NOT EXISTS achievements
                 (user_id INTEGER,
                  achievement_name TEXT,
                  achieved_date TIMESTAMP,
                  reward_claimed BOOLEAN DEFAULT FALSE,
                  PRIMARY KEY (user_id, achievement_name),
                  FOREIGN KEY (user_id) REFERENCES users(user_id))''')

    # Tabla de construcciones
    c.execute('''CREATE TABLE IF NOT EXISTS constructions
                 (construction_id INTEGER PRIMARY KEY AUTOINCREMENT,
                  user_id INTEGER,
                  building_type TEXT,
                  start_date TIMESTAMP,
                  completion_date TIMESTAMP,
                  status TEXT,
                  progress INTEGER DEFAULT 0,
                  FOREIGN KEY (user_id) REFERENCES users(user_id))''')

    # Tabla de eventos especiales
    c.execute('''CREATE TABLE IF NOT EXISTS special_events
                 (event_id INTEGER PRIMARY KEY AUTOINCREMENT,
                  event_name TEXT,
                  start_date TIMESTAMP,
                  end_date TIMESTAMP,
                  description TEXT,
                  reward_type TEXT,
                  reward_amount INTEGER)''')

    # Tabla de mensajes diarios
    c.execute('''CREATE TABLE IF NOT EXISTS daily_messages
                 (message_id INTEGER PRIMARY KEY AUTOINCREMENT,
                  user_id INTEGER,
                  message_date DATE,
                  message_type TEXT,
                  content TEXT,
                  FOREIGN KEY (user_id) REFERENCES users(user_id))''')

    conn.commit()
    conn.close()

# Inicializar base de datos
init_database()

In [51]:
# Constantes del juego
NIVELES_CONSTRUCCION = {
    1: {
        "nombre": "Casa Básica",
        "costo": 100,
        "recompensa": 50,
        "tiempo_construccion": 300,  # 5 minutos
        "materiales_necesarios": {
            "madera": 10,
            "cemento": 5,
            "ladrillos": 20
        }
    },
    2: {
        "nombre": "Casa Mejorada",
        "costo": 250,
        "recompensa": 125,
        "tiempo_construccion": 600,  # 10 minutos
        "materiales_necesarios": {
            "madera": 20,
            "cemento": 10,
            "ladrillos": 40
        }
    },
    3: {
        "nombre": "Casa Familiar",
        "costo": 500,
        "recompensa": 250,
        "tiempo_construccion": 1200,  # 20 minutos
        "materiales_necesarios": {
            "madera": 40,
            "cemento": 20,
            "ladrillos": 80
        }
    },
    4: {
        "nombre": "Casa Premium",
        "costo": 1000,
        "recompensa": 500,
        "tiempo_construccion": 1800,  # 30 minutos
        "materiales_necesarios": {
            "madera": 80,
            "cemento": 40,
            "ladrillos": 160
        }
    },
    5: {
        "nombre": "Mansión",
        "costo": 2000,
        "recompensa": 1000,
        "tiempo_construccion": 3600,  # 1 hora
        "materiales_necesarios": {
            "madera": 160,
            "cemento": 80,
            "ladrillos": 320
        }
    }
}

# Sistema de logros
LOGROS = {
    "constructor_novato": {
        "nombre": "Constructor Novato",
        "descripcion": "Construye tu primera casa",
        "recompensa": 100,
        "icono": "🏠"
    },
    "constructor_experto": {
        "nombre": "Constructor Experto",
        "descripcion": "Construye 10 casas",
        "recompensa": 500,
        "icono": "🏘️"
    },
    "maestro_constructor": {
        "nombre": "Maestro Constructor",
        "descripcion": "Construye 50 casas",
        "recompensa": 2000,
        "icono": "🏰"
    },
    "donador_bronce": {
        "nombre": "Donador Bronce",
        "descripcion": "Realiza tu primera donación",
        "recompensa": 200,
        "icono": "🥉"
    },
    "donador_plata": {
        "nombre": "Donador Plata",
        "descripcion": "Acumula 1000 en donaciones",
        "recompensa": 1000,
        "icono": "🥈"
    },
    "donador_oro": {
        "nombre": "Donador Oro",
        "descripcion": "Acumula 5000 en donaciones",
        "recompensa": 5000,
        "icono": "🥇"
    }
}

# Eventos especiales
EVENTOS_ESPECIALES = {
    "lluvia_monedas": {
        "nombre": "Lluvia de Monedas",
        "descripcion": "¡Monedas extra por cada construcción!",
        "multiplicador": 2,
        "duracion": 3600  # 1 hora
    },
    "construccion_rapida": {
        "nombre": "Construcción Rápida",
        "descripcion": "¡Tiempo de construcción reducido!",
        "multiplicador": 0.5,
        "duracion": 3600  # 1 hora
    },
    "materiales_abundantes": {
        "nombre": "Materiales Abundantes",
        "descripcion": "¡Menos materiales necesarios!",
        "multiplicador": 0.7,
        "duracion": 3600  # 1 hora
    }
}

In [56]:
class DreamBuilderGame:
    def __init__(self, user_id):
        self.user_id = user_id
        self.conn = sqlite3.connect('dreambuilder.db')
        self.cursor = self.conn.cursor()
        self.load_user_data()

    def load_user_data(self):
        self.cursor.execute("""
            SELECT level, coins, experience, houses_built
            FROM users WHERE user_id = ?
        """, (self.user_id,))
        data = self.cursor.fetchone()
        if data:
            self.level, self.coins, self.experience, self.houses_built = data
        else:
            self.level = 1
            self.coins = 0
            self.experience = 0
            self.houses_built = 0

    def start_construction(self, building_type):
        if building_type in NIVELES_CONSTRUCCION:
            cost = NIVELES_CONSTRUCCION[building_type]["costo"]
            if self.coins >= cost:
                self.coins -= cost
                self.cursor.execute("""
                    INSERT INTO constructions (user_id, building_type, start_date, status)
                    VALUES (?, ?, datetime('now'), 'in_progress')
                """, (self.user_id, building_type))
                self.conn.commit()
                return True, "¡Construcción iniciada!"
            return False, "No tienes suficientes monedas."
        return False, "Tipo de construcción no válido."

    def check_construction_progress(self):
        self.cursor.execute("""
            SELECT building_type, start_date, progress
            FROM constructions
            WHERE user_id = ? AND status = 'in_progress'
            ORDER BY start_date DESC LIMIT 1
        """, (self.user_id,))
        construction = self.cursor.fetchone()

        if construction:
            building_type, start_date, progress = construction
            total_time = NIVELES_CONSTRUCCION[building_type]["tiempo_construccion"]
            elapsed_time = (datetime.now() - datetime.strptime(start_date, '%Y-%m-%d %H:%M:%S')).total_seconds()
            current_progress = min(100, int((elapsed_time / total_time) * 100))

            if current_progress >= 100:
                self.complete_construction(building_type)
                return 100, "¡Construcción completada!"

            return current_progress, self.generate_progress_bar(current_progress)

        return 0, "No hay construcciones en progreso."

    def generate_progress_bar(self, progress):
        bar_length = 20
        filled = int(bar_length * progress / 100)
        bar = '█' * filled + '░' * (bar_length - filled)
        return f"Progreso: [{bar}] {progress}%"

    def complete_construction(self, building_type):
        reward = NIVELES_CONSTRUCCION[building_type]["recompensa"]
        self.coins += reward
        self.houses_built += 1
        self.experience += reward // 2

        # Actualizar base de datos
        self.cursor.execute("""
            UPDATE users
            SET coins = ?, houses_built = ?, experience = ?
            WHERE user_id = ?
        """, (self.coins, self.houses_built, self.experience, self.user_id))

        self.cursor.execute("""
            UPDATE constructions
            SET status = 'completed', completion_date = datetime('now')
            WHERE user_id = ? AND status = 'in_progress'
        """, (self.user_id,))

        self.conn.commit()
        self.check_achievements()

    def check_achievements(self):
        for achievement_id, achievement in LOGROS.items():
            if achievement_id == "constructor_novato" and self.houses_built >= 1:
                self.award_achievement(achievement_id)
            elif achievement_id == "constructor_experto" and self.houses_built >= 10:
                self.award_achievement(achievement_id)
            elif achievement_id == "maestro_constructor" and self.houses_built >= 50:
                self.award_achievement(achievement_id)

    def award_achievement(self, achievement_id):
        self.cursor.execute("""
            INSERT OR IGNORE INTO achievements (user_id, achievement_name, achieved_date)
            VALUES (?, ?, datetime('now'))
        """, (self.user_id, achievement_id))
        self.conn.commit()

In [58]:
# Comando /start
def start(update: Update, context: CallbackContext):
    user = update.effective_user
    chat_id = update.effective_chat.id

    # Registrar al usuario en la base de datos si no existe
    conn = sqlite3.connect('dreambuilder.db')
    cursor = conn.cursor()
    cursor.execute("""
        INSERT OR IGNORE INTO users (user_id, username, first_name, last_name)
        VALUES (?, ?, ?, ?)
    """, (user.id, user.username, user.first_name, user.last_name))
    conn.commit()
    conn.close()

    # Mensaje de bienvenida
    update.message.reply_text(
        f"👋 ¡Hola, {user.first_name}! Bienvenido a *DreamBuilder: Construyendo Esperanzas*.\n\n"
        "🏠 Aquí puedes construir casas virtuales, ganar logros y contribuir a la campaña 'Casas para Todos'.\n\n"
        "Usa /help para ver los comandos disponibles.",
        parse_mode=ParseMode.MARKDOWN
    )

# Comando /help
def help_command(update: Update, context: CallbackContext):
    update.message.reply_text(
        "📜 *Comandos disponibles:*\n\n"
        "/start - Inicia el bot\n"
        "/perfil - Muestra tu perfil\n"
        "/juego - Accede al mini-juego\n"
        "/donar - Realiza una donación\n"
        "/eventos - Consulta eventos especiales\n"
        "/referir - Obtén tu código de referido\n"
        "/logros - Consulta tus logros\n"
        "/notificar - (Admin) Envía notificaciones a todos los usuarios\n"
        "/privacidad - Consulta las políticas de privacidad\n",
        parse_mode=ParseMode.MARKDOWN
    )

# Comando /perfil
def perfil(update: Update, context: CallbackContext):
    user = update.effective_user
    conn = sqlite3.connect('dreambuilder.db')
    cursor = conn.cursor()

    # Obtener datos del usuario
    cursor.execute("""
        SELECT level, coins, experience, houses_built, total_donations
        FROM users WHERE user_id = ?
    """, (user.id,))
    data = cursor.fetchone()
    conn.close()

    if data:
        level, coins, experience, houses_built, total_donations = data
        update.message.reply_text(
            f"👤 *Tu Perfil:*\n\n"
            f"🏅 Nivel: {level}\n"
            f"💰 Monedas: {coins}\n"
            f"⭐ Experiencia: {experience}\n"
            f"🏠 Casas construidas: {houses_built}\n"
            f"🎁 Donaciones totales: ${total_donations:.2f}\n",
            parse_mode=ParseMode.MARKDOWN
        )
    else:
        update.message.reply_text("⚠️ No se encontraron datos de tu perfil. Usa /start para registrarte.")

In [60]:
# Comando /donar
def donar(update: Update, context: CallbackContext):
    user = update.effective_user

    # Verificar si se proporcionó un monto
    if len(context.args) == 0:
        update.message.reply_text("Por favor, proporciona un monto para donar. Ejemplo: /donar 10")
        return

    try:
        monto = float(context.args[0])
    except ValueError:
        update.message.reply_text("⚠️ El monto debe ser un número. Ejemplo: /donar 10")
        return

    conn = sqlite3.connect('dreambuilder.db')
    cursor = conn.cursor()

    # Registrar la donación
    cursor.execute("""
        INSERT INTO donations (user_id, amount, timestamp, payment_method, status)
        VALUES (?, ?, datetime('now'), 'manual', 'completed')
    """, (user.id, monto))
    conn.commit()

    # Actualizar monedas del usuario
    monedas_ganadas = int(monto * 10)  # 1 USD = 10 monedas
    cursor.execute("""
        UPDATE users
        SET coins = coins + ?, total_donations = total_donations + ?
        WHERE user_id = ?
    """, (monedas_ganadas, monto, user.id))
    conn.commit()
    conn.close()

    update.message.reply_text(
        f"🎉 ¡Gracias por tu donación de ${monto:.2f}! Se han añadido {monedas_ganadas} monedas a tu cuenta."
    )

In [62]:
# Comando /eventos
def eventos(update: Update, context: CallbackContext):
    conn = sqlite3.connect('dreambuilder.db')
    cursor = conn.cursor()

    # Obtener eventos activos
    cursor.execute("""
        SELECT event_name, description, reward_type, reward_amount, end_date
        FROM special_events
        WHERE end_date > datetime('now')
    """)
    eventos_activos = cursor.fetchall()
    conn.close()

    if eventos_activos:
        mensaje = "🎉 *Eventos Especiales Activos:*\n\n"
        for evento in eventos_activos:
            nombre, descripcion, tipo_recompensa, cantidad, fin = evento
            mensaje += (
                f"📌 *{nombre}*\n"
                f"📝 {descripcion}\n"
                f"🎁 Recompensa: {cantidad} {tipo_recompensa}\n"
                f"⏳ Finaliza: {fin}\n\n"
            )
    else:
        mensaje = "⚠️ No hay eventos especiales activos en este momento."

    update.message.reply_text(mensaje, parse_mode=ParseMode.MARKDOWN)

In [64]:
# Comando /referir
def referir(update: Update, context: CallbackContext):
    user = update.effective_user
    conn = sqlite3.connect('dreambuilder.db')
    cursor = conn.cursor()

    # Obtener el código de referido del usuario
    cursor.execute("SELECT referral_code FROM users WHERE user_id = ?", (user.id,))
    referral_code = cursor.fetchone()
    conn.close()

    if referral_code:
        update.message.reply_text(
            f"🎉 ¡Invita a tus amigos a unirse a DreamBuilder!\n\n"
            f"🔗 Tu código de referido: *{referral_code[0]}*\n\n"
            f"Por cada amigo que se registre con tu código, ambos recibirán recompensas exclusivas. "
            f"¡Comparte este código y ayuda a construir más casas!",
            parse_mode=ParseMode.MARKDOWN
        )
    else:
        update.message.reply_text("⚠️ No se pudo generar tu código de referido. Intenta nuevamente.")

In [66]:
# Comando /logros
def logros(update: Update, context: CallbackContext):
    user = update.effective_user
    conn = sqlite3.connect('dreambuilder.db')
    cursor = conn.cursor()

    # Obtener logros del usuario
    cursor.execute("""
        SELECT achievement_name, achieved_date
        FROM achievements
        WHERE user_id = ?
    """, (user.id,))
    logros = cursor.fetchall()
    conn.close()

    if logros:
        mensaje = "🏆 *Tus Logros:*\n\n"
        for logro in logros:
            nombre, fecha = logro
            mensaje += f"✅ {LOGROS[nombre]['icono']} *{LOGROS[nombre]['nombre']}* - {fecha}\n"
    else:
        mensaje = "⚠️ No has desbloqueado ningún logro aún. ¡Sigue construyendo y donando para ganar logros!"

    update.message.reply_text(mensaje, parse_mode=ParseMode.MARKDOWN)

In [68]:
# Comando /notificar (solo para el creador)
def notificar(update: Update, context: CallbackContext):
    if update.effective_user.id != CREATOR_INFO["id"]:
        update.message.reply_text("⚠️ No tienes permiso para usar este comando.")
        return

    if len(context.args) == 0:
        update.message.reply_text("Por favor, proporciona un mensaje para enviar. Ejemplo: /notificar ¡Hola a todos!")
        return

    mensaje = " ".join(context.args)
    conn = sqlite3.connect('dreambuilder.db')
    cursor = conn.cursor()

    # Obtener todos los usuarios
    cursor.execute("SELECT user_id FROM users")
    usuarios = cursor.fetchall()
    conn.close()

    for usuario in usuarios:
        context.bot.send_message(chat_id=usuario[0], text=mensaje)

    update.message.reply_text("✅ Notificación enviada a todos los usuarios.")

In [70]:
# Enviar mensajes automáticos
def enviar_mensajes_diarios():
    conn = sqlite3.connect('dreambuilder.db')
    cursor = conn.cursor()

    # Obtener usuarios
    cursor.execute("SELECT user_id FROM users")
    usuarios = cursor.fetchall()
    conn.close()

    for usuario in usuarios:
        context.bot.send_message(chat_id=usuario[0], text="🌟 ¡No olvides construir tu casa hoy!")

In [74]:
# Sistema de autoguardado
def auto_save():
    while True:
        IPython.display.display(IPython.display.Javascript('google.colab.kernel.save()'))
        time.sleep(300)  # Guardar cada 60 minutos

In [ ]:
# Manejo de errores
def error_handler(update: Update, context: CallbackContext):
    """Log Errors caused by Updates."""
    logger.warning(f'Update "{update}" caused error "{context.error}"')
    try:
        if update and update.effective_message:
            update.effective_message.reply_text(
                "❌ Ocurrió un error. El bot intentará recuperarse automáticamente."
            )
    except Exception as e:
        logger.error(f"Error en el manejador de errores: {e}")

# Configuración del bot
updater = Updater(TOKEN)

# Registrar comandos
updater.dispatcher.add_handler(CommandHandler("start", start))
updater.dispatcher.add_handler(CommandHandler("help", help_command))
updater.dispatcher.add_handler(CommandHandler("perfil", perfil))
updater.dispatcher.add_handler(CommandHandler("donar", donar))
updater.dispatcher.add_handler(CommandHandler("eventos", eventos))
updater.dispatcher.add_handler(CommandHandler("referir", referir))
updater.dispatcher.add_handler(CommandHandler("logros", logros))
updater.dispatcher.add_handler(CommandHandler("notificar", notificar))

# Registrar el manejador de errores
updater.dispatcher.add_error_handler(error_handler)

# Iniciar el bot
print("🤖 Bot iniciado. Presiona Ctrl+C para detener.")
updater.start_polling()
updater.idle()

🤖 Bot iniciado. Presiona Ctrl+C para detener.
